# Dataset Overview
The dataset includes 1460 rows and 81 columns. Each row represents a house and each column represents a feature of the house. The target column is 'SalePrice'. Here are 20/81 of the key features:

- **SalePrice (Target Variable)**: This column is essential for your analysis as it represents the property's sale price that you're trying to predict.

- **OverallQual**: This column represents the overall material and finish quality of the property. It can be a significant factor in determining the sale price.

- **GrLivArea**: The above-grade living area in square feet can have a significant impact on the property's value.

- **GarageCars**: The size of the garage in terms of car capacity can be an important feature affecting the property's price.

- **TotalBsmtSF**: The total square feet of the basement area can also play a crucial role in determining the property's value.

- **1stFlrSF**: The square footage of the first floor can be a valuable feature in estimating the property's price.

- **FullBath**: The number of full bathrooms above grade can be an influential factor for potential buyers.

- **TotRmsAbvGrd**: The total number of rooms above grade (excluding bathrooms) can provide insight into the property's size and functionality.

- **YearBuilt**: The original construction date of the property can affect its value, considering factors such as architectural style and historical significance.

- **YearRemodAdd**: The remodel date can indicate if any recent renovations or improvements were made, which can impact the property's price.

- **Neighborhood**: The physical location within Ames city limits can have a significant influence on property values based on factors like amenities, schools, and desirability.

- **MasVnrArea**: The masonry veneer area in square feet can add aesthetic value to the property and impact its price.

- **GarageArea**: The size of the garage in square feet is another essential factor to consider.

- **BsmtFinSF1**: The type 1 finished square feet of the basement area can contribute to the property's value.

- **OpenPorchSF**: The square footage of the open porch area can enhance the property's appeal and potentially increase its value.

- **WoodDeckSF**: The area of the wood deck in square feet can be an attractive feature for buyers and impact the property's price.

- **2ndFlrSF**: The square footage of the second floor can provide additional living space and influence the property's value.

- **HalfBath**: The number of half baths above grade can be a useful feature for buyers and affect the property's price.

- **LotArea**: The size of the lot in square feet can be a relevant factor in determining the property's value.

- **KitchenQual**: The quality of the kitchen can significantly impact the property's desirability and price.,

- **...**

In [104]:
import pandas as pd

file_path = 'data/house_pricing_train.csv'

original_df = pd.read_csv(file_path)

print(original_df.head())
print("Number of rows:", original_df.shape[0])

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCondition  SalePrice  
0   2008        WD   

In [105]:
original_df.info()
# listing categorical and numerical columns
categorical_columns = original_df.select_dtypes(include=['object']).columns
numerical_columns = original_df.select_dtypes(exclude=['object']).columns
print("Categorical columns:", categorical_columns)
print("Numerical columns:", numerical_columns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [106]:
set(original_df['YearRemodAdd'])

{1950,
 1951,
 1952,
 1953,
 1954,
 1955,
 1956,
 1957,
 1958,
 1959,
 1960,
 1961,
 1962,
 1963,
 1964,
 1965,
 1966,
 1967,
 1968,
 1969,
 1970,
 1971,
 1972,
 1973,
 1974,
 1975,
 1976,
 1977,
 1978,
 1979,
 1980,
 1981,
 1982,
 1983,
 1984,
 1985,
 1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010}

-----------------------------------------------------------------------
Next we find the categorical variables among the numerical columns. We can identify categorical variables by checking the ratio of unique values to the total number of values in a column. If this ratio is below a certain threshold, we can consider the column as categorical. In this case, we set the threshold to 0.05 (5%).
And then change the data type of these columns to 'object'.

In [107]:
data_types = original_df.dtypes

# Identify columns with numeric data types
numeric_columns = data_types[data_types != 'object'].index.tolist()

# Identify categorical variables among the numeric columns
categorical_vars = []
for column in numeric_columns:
    unique_values = original_df[column].nunique()
    total_values = len(original_df[column])
    if unique_values / total_values < 0.05:  # Set a threshold for categorical variables
        categorical_vars.append(column)

# Print the categorical variables
print("Categorical Variables (Numeric):")
print(categorical_vars)  

# Change the data type of these columns to 'object'
original_df[categorical_vars] = original_df[categorical_vars].astype('object')

Categorical Variables (Numeric):
['MSSubClass', 'OverallQual', 'OverallCond', 'YearRemodAdd', 'LowQualFinSF', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', '3SsnPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']


----------------------------------------------------------------------- 
Remove id column because it is not useful for the analysis.

In [108]:
original_df.drop('Id', axis=1, inplace=True)
original_df.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

-----------------------------------------------------------------------
Next, let's find out more information about the numerical variables:

In [109]:
original_df.describe()

,LotFrontage,LotArea,YearBuilt,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,GarageYrBlt,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,ScreenPorch,SalePrice
count,1201.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,70.049958,10516.828082,1971.267808,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,1515.463699,1978.506164,472.980137,94.244521,46.660274,21.954110,15.060959,180921.195890
std,24.284752,9981.264932,30.202904,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,525.480383,24.689725,213.804841,125.338794,66.256028,61.119149,55.757415,79442.502883
min,21.000000,1300.000000,1872.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,334.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,34900.000000
25%,59.000000,7553.500000,1954.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,1129.500000,1961.000000,334.500000,0.000000,0.000000,0.000000,0.000000,129975.000000
50%,69.000000,9478.500000,1973.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,1464.000000,1980.000000,480.000000,0.000000,25.000000,0.000000,0.000000,163000.000000
75%,80.000000,11601.500000,2000.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,1776.750000,2002.000000,576.000000,168.000000,68.000000,0.000000,0.000000,214000.000000
max,313.000000,215245.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,5642.000000,2010.000000,1418.000000,857.000000,547.000000,552.000000,480.000000,755000.000000


LotFrontage has a range of values from 21 to 313.0,
MasVnrArea has a range of values from 0 to 1600.0,
GarageYrBlt has a range of values from 1900 to 2010.
...

These columns have outliers, which can affect the model's performance. We will handle these outliers in the data preprocessing step.


-----------------------------------------------------------------------
## Data Cleaning

At first, we make a copy of the original dataframe to keep the original data unchanged.

In [110]:
df = original_df.copy()

# Data duplication

In [111]:
dup_ind = df.duplicated()
df[dup_ind]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice


There is no duplicated row in the dataset.

# Missing values

In [112]:
df.isnull().sum()*100/df.shape[0]

MSSubClass        0.000000
MSZoning          0.000000
LotFrontage      17.739726
LotArea           0.000000
Street            0.000000
                   ...    
MoSold            0.000000
YrSold            0.000000
SaleType          0.000000
SaleCondition     0.000000
SalePrice         0.000000
Length: 80, dtype: float64

We can see that there are many columns with missing values.

In [113]:
print(set(df['Alley']))
print(set(df['PoolQC']))
print(set(df['Fence']))
print(set(df['MiscFeature']))
print(set(df['GarageCond']))
print(set(df['GarageQual']))
print(set(df['GarageFinish']))
print(set(df['GarageType']))
print(set(df['FireplaceQu']))
print(set(df['BsmtQual']))
print(set(df['BsmtCond']))
print(set(df['BsmtExposure']))
print(set(df['BsmtFinType1']))
print(set(df['BsmtFinType2']))
print(set(df['MasVnrType']))
print(set(df['Electrical']))


{'Pave', nan, 'Grvl'}
{'Gd', nan, 'Fa', 'Ex'}
{'GdWo', 'MnPrv', 'GdPrv', 'MnWw', nan}
{'Gar2', 'TenC', 'Shed', nan, 'Othr'}
{nan, 'Ex', 'Po', 'Gd', 'Fa', 'TA'}
{nan, 'Ex', 'Po', 'Gd', 'Fa', 'TA'}
{'RFn', 'Fin', nan, 'Unf'}
{'BuiltIn', '2Types', 'CarPort', 'Detchd', 'Basment', nan, 'Attchd'}
{'Ex', 'Fa', 'Po', 'Gd', nan, 'TA'}
{'Ex', 'Fa', 'Gd', nan, 'TA'}
{'Fa', 'Po', 'Gd', nan, 'TA'}
{'No', 'Mn', 'Gd', nan, 'Av'}
{'LwQ', 'GLQ', 'BLQ', 'Rec', 'ALQ', nan, 'Unf'}
{'LwQ', 'GLQ', 'BLQ', 'Rec', 'ALQ', nan, 'Unf'}
{'BrkFace', nan, 'BrkCmn', 'Stone'}
{'SBrkr', 'Mix', 'FuseP', 'FuseA', nan, 'FuseF'}


because 'nan' in these columns is not missing values, it is a category. We will replace 'nan' with 'No' to distinguish it from missing values.

In [114]:
df['Alley'] = df['Alley'].fillna('No')
df['PoolQC'] = df['PoolQC'].fillna('No')
df['Fence'] = df['Fence'].fillna('No')
df['MiscFeature'] = df['MiscFeature'].fillna('No')
df['GarageCond'] = df['GarageCond'].fillna('No')
df['GarageQual'] = df['GarageQual'].fillna('No')
df['GarageFinish'] = df['GarageFinish'].fillna('No')
df['GarageType'] = df['GarageType'].fillna('No')
df['FireplaceQu'] = df['FireplaceQu'].fillna('No')
df['BsmtQual'] = df['BsmtQual'].fillna('No')
df['BsmtCond'] = df['BsmtCond'].fillna('No')
df['BsmtExposure'] = df['BsmtExposure'].fillna('No')
df['BsmtFinType1'] = df['BsmtFinType1'].fillna('No')
df['BsmtFinType2'] = df['BsmtFinType2'].fillna('No')
df['MasVnrType'] = df['MasVnrType'].fillna('No')
df['Electrical'] = df['Electrical'].fillna('No')

For other numerical columns, we will replace missing values with the median value of the column.

In [115]:
df['LotFrontage'] = df['LotFrontage'].fillna(df['LotFrontage'].median())
df['MasVnrArea'] = df['MasVnrArea'].fillna(df['MasVnrArea'].median())
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(df['GarageYrBlt'].median())

Let's check the missing values again.

In [116]:
df.isnull().sum()*100/df.shape[0]

MSSubClass       0.0
MSZoning         0.0
LotFrontage      0.0
LotArea          0.0
Street           0.0
                ... 
MoSold           0.0
YrSold           0.0
SaleType         0.0
SaleCondition    0.0
SalePrice        0.0
Length: 80, dtype: float64